# LangChain Tool Call과 Agent

###핵심 개념 정리

- Tool: LLM이 필요할 때 호출하는 외부 기능(예: 웹 검색, 계산기, DB 질의).

- Tool Call: 모델이 “이 질문은 검색이 필요하네”처럼 도구 호출 의사결정을 내려 실제로 도구를 실행하는 단계.

- Agent: “언제 어떤 도구를 쓸지” 스스로 판단해 도구 호출 → 결과 반영 → 답변 생성을 반복 실행하는 실행자.

- bind_tools: LLM에 사용할 도구들을 등록해 도구 호출 가능한 모델로 만드는 LangChain 방식.

### 단계 1. AI 모델 준비
- Google Gemini와 OpenAI GPT 같은 AI 모델을 불러옵니다.  
- 동시에 요청 횟수를 제한하는 ‘속도 제한 장치(Rate Limiter)’도 둡니다.  
  → 고객센터에 전화를 한꺼번에 너무 많이 걸리지 않게 막는 역할과 비슷합니다.

### 단계 2. 대화의 뼈대 만들기
- 사람 질문(HumanMessage), 시스템 지침(SystemMessage), AI 답변(AIMessage)을 정리해 대화 기록을 만듭니다.  
- 이렇게 하면 AI가 대화 맥락을 이해하고 친절한 톤으로 답할 수 있습니다.

### 단계 3. 프롬프트 체인 만들기
- “프롬프트”는 AI에게 지시하는 대본 같은 것입니다.  
- 이를 LLM과 연결해 체인으로 만들면, 질문이 들어올 때마다 같은 규칙으로 대답합니다.

### 단계 4. 외부 검색 도구 연결
- Tavily 검색 같은 도구를 붙여서, AI가 최신 정책이나 정보를 직접 찾아보고 답하게 합니다.  
- 예: “현재 환불 규정 알려줘” → AI가 도구를 써서 검색 후 답변.

### 단계 5. 사용자 정의 도구 추가
- 숫자를 합산하거나, 문장을 뒤집는 등 직접 만든 도구를 AI가 활용할 수 있습니다.  
- 이렇게 하면 단순 계산이나 텍스트 가공도 AI가 자동 처리합니다.

### 단계 6. 최종 워크플로우
- 사용자의 질문 → AI가 필요시 도구 호출 → 결과 반영 → 최종 답변 생성.  
- 즉, **AI + 규칙 + 검색도구 + 맞춤도구**가 합쳐져 고객센터 상담을 자동화하는 구조입니다.


In [1]:
!pip3 install -U langchain langchain_core langchain_community langchain_google_genai tavily-python
# 👉 필요한 라이브러리 설치 (LangChain + Google Gemini + Tavily 검색 툴)

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.0 MB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 457 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.5 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 463 kB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 396 kB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 5.1 MB/s eta 0:00:011
     |████████████████████████████████| 174 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 490 kB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 7.3 MB/s eta 0:00:01
     |████████████████████████████████|

In [4]:
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI
# 👉 요청 제한기(Rate Limiter)와 Gemini 모델 불러오기
# from langchain_openai import ChatOpenAI
# 👉 필요시 OpenAI 모델도 불러와 사용할 수 있음 (지금은 주석 처리)
# 고객지원 시나리오에서도 동일한 호출 제한 가정 (분당 10회)
request_throttle = InMemoryRateLimiter(
    requests_per_second=0.167,  # 👉 초당 약 0.167회 → 분당 10회 제한
    check_every_n_seconds=0.1,  # 👉 0.1초마다 요청 횟수 체크
    max_bucket_size=10,         # 👉 최대 10개까지 한 번에 처리 가능
)

# Rate Limiter를 적용한 LLM 인스턴스
cs_llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",  # 👉 사용할 Gemini 모델 지정
    rate_limiter=request_tßrottle, # 👉 위에서 만든 요청 제한기 적용
)

# 간단 동작 확인용 테스트 질문(도메인: 고객지원)
cs_llm.invoke("고객 응대란 무엇인지 한 문장으로 설명해줘!")
# 👉 모델이 정상 작동하는지 테스트


AIMessage(content='고객 응대란 고객의 문의, 불만, 요청 등에 대해 기업이나 조직이 적절하고 신속하게 대응하여 고객 만족도를 높이는 모든 활동을 의미합니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-exp', 'safety_ratings': []}, id='run--d0281be2-54a9-44f9-b4c2-832ace8958b4-0', usage_metadata={'input_tokens': 15, 'output_tokens': 39, 'total_tokens': 54, 'input_token_details': {'cache_read': 0}})

### LLM이 답변하는 방법

In [5]:
# 1. HumanMessage를 이용한 방법
# LangChain에서 대화 메시지 타입 세 가지를 불러옵니다:
# - HumanMessage: 사용자의 질문
# - SystemMessage: 답변 규칙·조건
# - AIMessage: AI가 생성한 응답
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

support_system_msg = SystemMessage(content="""아래 [문의]에 대해 적절하게 답하시오.
답변은 2문장 이내로 하고, 친절하고 정확한 어조로 답하세요.""")
# 👉 시스템 메시지: "답변은 2문장 이내, 친절하게" 규칙 설정

customer_msg = HumanMessage(content="환불 절차를 간단히 설명해줘!")
# 👉 고객이 실제로 묻는 질문 입력

conversation_log = [support_system_msg, customer_msg]
# 👉 대화 기록을 리스트로 구성

cs_response = cs_llm.invoke(conversation_log)
cs_response
# 👉 대화 기록을 모델에 전달하고 응답 받기

# Chat 모델의 출력은 AIMessage라는 형식으로 변환됩니다.
print(cs_response.content)
# 👉 실제 답변 내용 출력


환불은 구매하신 제품의 반송이 완료되면 영업일 기준 3~5일 이내에 결제하신 수단으로 처리됩니다. 자세한 내용은 저희 웹사이트의 환불 정책 페이지를 참고해 주시면 감사하겠습니다.


In [6]:
# 2. ChatPromptTemplate을 이용한 방법
from langchain_core.prompts import ChatPromptTemplate
# 👉 대화 프롬프트(질문 틀) 생성 도구

# 입력 변수가 {inquiry}인 템플릿 (도메인: 고객지원)
support_prompt = ChatPromptTemplate([
    ('system', '''아래 [문의]에 답변하시오.
답변은 2문장 이내로 하고, 친절하고 정확한 어조로 답하세요.'''),
    ('user', '''[문의]: {inquiry}''')
])
# 👉 system 규칙 + user 질문을 템플릿으로 정의

support_prompt
# 👉 템플릿 객체 확인

# 프롬프트와 LLM(cs_llm)을 | 로 연결하여 체인 구성
support_chain = support_prompt | cs_llm
support_chain
# 👉 프롬프트와 모델을 연결해 하나의 체인으로 사용

cs_resp = support_chain.invoke({'inquiry': "고객 FAQ와 맞춤 스크립트의 차이가 뭐야?"})
print(cs_resp.content)
# 👉 템플릿에 질문 전달 → 답변 생성


고객 FAQ는 자주 묻는 질문에 대한 답변 모음집인 반면, 맞춤 스크립트는 특정 상황이나 고객 문의에 맞춰 미리 준비된 대화 흐름입니다. 따라서 맞춤 스크립트는 FAQ보다 더 구체적이고 상황에 맞는 답변을 제공할 수 있습니다.
